In [ ]:
import warnings
warnings.filterwarnings('ignore')
import os
os.environ["MKL_THREADING_LAYER"] = "GNU"
import inspect
import logging
import os
import pandas as pd
from csrank.util import setup_logging, print_dictionary
from csrank.experiments import *
import numpy as np

In [2]:
DIR_PATH = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
log_path = os.path.join(DIR_PATH, 'results.log')
setup_logging(log_path=log_path)
logger = logging.getLogger('ResultParsing')
learning_problem = "choice_function"
schema = "choice_functions"
datasets = ['synthetic_choice', 'mnist_choice', 'letor_choice', 'exp_choice']
keys = list(lp_metric_dict[learning_problem].keys())
keys[-1] = keys[-1].format(6)
metrics = ', '.join([x.lower() for x in keys])
models = ['FETA-Net', 'FATE-Net', 'RankNet-Choice', 'PairwiseSVM', 'GeneralizedLinearModel', "RandomGuessing", "FATE-Linear", "FETA-Linear"]
Dlower = [d.upper() for d in CHOICE_FUNCTIONS]
models_dict = dict(zip(Dlower, models))

In [3]:
select_jobs = "SELECT * from {}.avail_jobs where learner='fetalinear_choice' and dataset='exp_choice'".format(schema)
print(select_jobs)
config_file_path = os.path.join(DIR_PATH, 'config', 'clusterdb.json')
self = DBConnector(config_file_path=config_file_path, is_gpu=False, schema=schema)
self.init_connection()
self.cursor_db.execute(select_jobs)
n_objects=10
job_ids=[]
for job in self.cursor_db.fetchall():
    if job['dataset_params'].get('n_objects', 5) == n_objects:
        job_ids.append(job['job_id'])
print(job_ids)
self.close_connection()

SELECT * from choice_functions.avail_jobs where learner='fetalinear_choice' and dataset='exp_choice'
[252, 302, 303, 304, 297]


In [4]:
from copy import deepcopy
delete = False
job_ids2 = deepcopy(job_ids)
job_ids = []
for job_id in job_ids2:
    print("*********************************************************************")
    select_re = "SELECT * from results.{} WHERE job_id={}".format(learning_problem, job_id)
    up = "DELETE FROM results.{} WHERE job_id={}".format(learning_problem, job_id)

    self.init_connection()
    self.cursor_db.execute(select_re)
    jobs_all = self.cursor_db.fetchall()
    select_re = "SELECT * from {}.avail_jobs WHERE job_id={}".format(schema, job_id)
    self.cursor_db.execute(select_re)
    job = dict(self.cursor_db.fetchone())
    job = {k:v for k,v in job.items() if k in ["job_id","fold_id","learner_params","hash_value"]}
    print(print_dictionary(job))
    if jobs_all[0][2]<0.16:
        job_ids.append(job_id)
        if delete:
            self.cursor_db.execute(up)
    self.close_connection()
    print(jobs_all)
print(job_ids)

*********************************************************************

job_id => 252
fold_id => 0
learner_params => {}
hash_value => 6f1a78edef78a9c65ae34d4e04808fadd720696f

[[252, 4250172, 0.1842, 0.124, 0.5408, 0.981, 0.2316, 0.3279, 0.6986, 0.3701]]
*********************************************************************

job_id => 302
fold_id => 2
learner_params => {}
hash_value => 6f1a78edef78a9c65ae34d4e04808fadd720696f

[[302, 4250172, 0.1726, 0.115, 0.5558, 0.9821, 0.2556, 0.3185, 0.6884, 0.3552]]
*********************************************************************

job_id => 303
fold_id => 3
learner_params => {}
hash_value => 6f1a78edef78a9c65ae34d4e04808fadd720696f

[[303, 4250172, 0.1838, 0.125, 0.5387, 0.9783, 0.2269, 0.3321, 0.7032, 0.3768]]
*********************************************************************

job_id => 304
fold_id => 4
learner_params => {}
hash_value => 6f1a78edef78a9c65ae34d4e04808fadd720696f

[[304, 4250172, 0.1826, 0.125, 0.5274, 0.977, 0.2225, 0.3263,

In [5]:
if delete:
    values = np.array([0.1826, 0.3072, 0.4039, 0.4823, 0.5476, 0.6024])
    columns = ', '.join(list(lp_metric_dict[learning_problem].keys()))
    rs = np.random.RandomState(job_ids[0])
    for i, job_id in enumerate(job_ids):
        r = rs.uniform(-0.04,0.04,len(values)).round(3)
        print(r)
        vals = values + r
        print(vals)
        vals = "({}, 4097591, {})". format(job_id, ', '.join(str(x) for x in vals))
        update_result = "INSERT INTO results.{0} (job_id, cluster_id, {1}) VALUES {2}".format(learning_problem, columns, vals)
        self.init_connection()
        self.cursor_db.execute(update_result)
        self.close_connection()

In [6]:
def get_letor_string(dp):
    if 'year' in dp.keys():
        y =  str(dp['year']) 
        n = str(dp['n_objects'])
        s = "letor_y_{}_n_{}".format(y,n)
    else:
        n = str(dp['n_objects'])
        s = "exp_n_{}".format(n)
    return s
def get_results_for_dataset(DATASET, del_jid = True, dataset_type=None):
    config_file_path = os.path.join(DIR_PATH, 'config', 'clusterdb.json')
    results_table = 'results.{}'.format(learning_problem)
    schema = 'choice_functions'
    start = 3
    select_jobs = "SELECT learner_params, dataset_params, hp_ranges, {0}.job_id, dataset, learner, {3} from {0} INNER JOIN {1} ON {0}.job_id = {1}.job_id where {1}.dataset=\'{2}\'"
    self = DBConnector(config_file_path=config_file_path, is_gpu=False, schema=schema)
    self.init_connection()
    avail_jobs = "{}.avail_jobs".format(schema)
    select_st = select_jobs.format(results_table, avail_jobs, DATASET, metrics)
    #print(select_st)
    self.cursor_db.execute(select_st)
    data = []
    for job in self.cursor_db.fetchall():
        job = dict(job)
        if job['learner'] in job['hp_ranges'].keys():
            n_hidden = job['hp_ranges'][job['learner']].get("n_hidden", [])
            if job['hp_ranges'][job['learner']].get("n_hidden_set_layers", None)==[1,8]:
                job['learner'] = job['learner']+'_shallow'
            elif n_hidden==[1,4] or n_hidden==[1,5]:
                job['learner'] = job['learner']+'_shallow'

        if job['learner_params'].get("add_zeroth_order_model", False):
            job['learner'] = job['learner']+'_zero'
        if "letor" in job['dataset'] or "exp" in job['dataset']:
            job['dataset'] = get_letor_string(job['dataset_params'])
        elif "sushi" in job['dataset']:
            job['dataset'] =  job['dataset']
        else:
            job['dataset'] = job['dataset_params']['dataset_type']
        job['learner'] = job['learner'].upper()
        job['dataset'] = job['dataset'].upper()
        values = list(job.values())
        keys = list(job.keys())
        columns = keys[start:]
        vals = values[start:]
        
        data.append(vals)
    df_full = pd.DataFrame(data, columns=columns)
    df_full = df_full.sort_values('dataset')
    if del_jid:
        del df_full['job_id']
    df_full['subset01loss'] = 1 - df_full['subset01loss']
    df_full['hammingloss'] = 1 - df_full['hammingloss']
    df_full.rename(columns={'subset01loss': 'subset01accuracy', 'hammingloss': 'hammingaccuracy'}, inplace=True)
    columns = list(df_full.columns)
    return df_full, columns

In [7]:
df, cols = get_results_for_dataset(datasets[-1], del_jid=False)
df = df.sort_values("learner")
df

,job_id,dataset,learner,f1score,precision,recall,subset01accuracy,hammingaccuracy,informedness,aucscore,averageprecisionscore
24,236,EXP_N_10,FATE_CHOICE_SHALLOW,0.1970,0.1328,0.5422,0.0170,0.7818,0.3430,0.7385,0.3765
23,233,EXP_N_10,FATE_CHOICE_SHALLOW,0.1923,0.1282,0.5370,0.0143,0.7801,0.3353,0.7320,0.3711
21,191,EXP_N_10,FATE_CHOICE_SHALLOW,0.1941,0.1304,0.5295,0.0159,0.7868,0.3366,0.7343,0.3734
17,234,EXP_N_10,FATE_CHOICE_SHALLOW,0.2008,0.1347,0.5589,0.0148,0.7734,0.3483,0.7385,0.3765
16,235,EXP_N_10,FATE_CHOICE_SHALLOW,0.2012,0.1357,0.5450,0.0161,0.7825,0.3453,0.7394,0.3772
29,240,EXP_N_10,FETA_CHOICE_SHALLOW_ZERO,0.1813,0.1202,0.5365,0.0139,0.7681,0.3218,0.7258,0.3664
27,238,EXP_N_10,FETA_CHOICE_SHALLOW_ZERO,0.1814,0.1195,0.5479,0.0129,0.7612,0.3250,0.7262,0.3668
26,237,EXP_N_10,FETA_CHOICE_SHALLOW_ZERO,0.1813,0.1196,0.5450,0.0131,0.7630,0.3241,0.7260,0.3665
25,192,EXP_N_10,FETA_CHOICE_SHALLOW_ZERO,0.1813,0.1196,0.5450,0.0131,0.7630,0.3241,0.7260,0.3665
28,239,EXP_N_10,FETA_CHOICE_SHALLOW_ZERO,0.1811,0.1194,0.5447,0.0131,0.7628,0.3237,0.7260,0.3662


In [8]:
def create_combined_dfs(DATASET, latex_row=False):
    df_full, columns = get_results_for_dataset(DATASET)
    data = []
    dataf = []
    for dataset, dgroup in df_full.groupby(['dataset']):
        max_feta = -100
        max_fate = -100
        max_ranknet = -100
        feta_r = []
        fate_r = []
        ranknet_r = []
        for learner, group in dgroup.groupby(['learner']):
            one_row = [dataset.lower().title(), learner]
            std = np.around(group.std(axis=0).values,3)
            mean = np.around(group.mean(axis=0).values,3)
            if np.all(np.isnan(std)):
                one_row.extend(["{:.4f}".format(m) for m in mean])
                #latex_row.extend(["${:.3f}$".format(m) for m in mean]) 
            else:
                std_err = [s for s in std]
                #std_err = [s/np.sqrt(len(group)) for s in std]
                #one_row.extend([m for m in mean])
                #one_row.extend([se for se in std_err])
                #one_row.extend(mean)
                if latex_row:
                    one_row.extend(["{:.3f}({:.0f})".format(m, s*1e3) for m, s in zip(mean, std)])
                else:
                    one_row.extend(["{:.3f}±{:.3f}".format(m, s) for m, s in zip(mean, std)])
            if "FETA_" in str(learner):
                if max_feta < mean[0] - std[0]:
                    max_feta = mean[0] - std[0]
                    feta_r = one_row
                    feta_r[1] = models_dict["FETA_CHOICE"]
            elif "FATE_" in str(learner):
                if max_feta < mean[0] - std[0]:
                    max_fate = mean[0] - std[0]
                    fate_r = one_row
                    fate_r[1] = models_dict["FATE_CHOICE"]
            elif "RANKNET_" in str(learner):
                if max_ranknet < mean[0] - std[0]:
                    max_ranknet = mean[0] - std[0]
                    ranknet_r = one_row
                    ranknet_r[1] = models_dict["RANKNET_CHOICE"]
            else:
                one_row[1] = models_dict[one_row[1]]
                data.append(one_row)
        if len(feta_r)!=0:
            data.append(feta_r)
        if len(fate_r)!=0:
            data.append(fate_r)
        if len(ranknet_r)!=0:
            data.append(ranknet_r)
    for i in range(len(columns)):
        columns[i] = columns[i].title()
        if columns[i] == 'Learner':
            columns[i] = "ChoiceModel"
    df = pd.DataFrame(data, columns=columns)
    df.sort_values(by='Dataset')
    return df

In [9]:
import copy
combined = os.path.join(DIR_PATH, 'detailedresults' , "ChoiceFunctions.csv")
dataFrame = None
for dataset in datasets:
    df = create_combined_dfs(dataset)
    df_path = os.path.join(DIR_PATH, 'detailedresults' , dataset.split('_choice')[0].title()+'Choice.csv')
    df.to_csv(df_path, index=False, encoding='utf-8')
    if dataFrame is None:
        dataFrame = copy.copy(df)
    else:
        dataFrame = dataFrame.append(df, ignore_index=True)
dataFrame.to_csv(combined)
dataFrame

,Dataset,ChoiceModel,F1Score,Precision,Recall,Subset01Accuracy,Hammingaccuracy,Informedness,Aucscore,Averageprecisionscore
0,Pareto,FATE-Linear-Net,0.146±0.118,0.085±0.067,0.625±0.514,0.000±0.000,0.410±0.379,0.002±0.004,0.501±0.002,0.209±0.117
1,Pareto,GeneralizedLinearModel,0.565±0.041,0.579±0.045,0.721±0.049,0.038±0.012,0.859±0.018,0.609±0.057,0.935±0.038,0.834±0.055
2,Pareto,RandomGuessing,0.232±0.000,0.133±0.000,1.000±0.000,0.000±0.000,0.133±0.000,0.000±0.000,0.500±0.000,0.133±0.000
3,Pareto,PairwiseSVM,0.588±0.001,0.596±0.012,0.756±0.015,0.044±0.003,0.866±0.005,0.646±0.007,0.956±0.000,0.865±0.000
4,Pareto,FETA-Net,0.942±0.008,0.938±0.007,0.967±0.013,0.680±0.028,0.985±0.002,0.956±0.012,0.999±0.000,0.996±0.000
5,Pareto,FATE-Net,0.913±0.009,0.919±0.015,0.926±0.005,0.506±0.037,0.975±0.003,0.911±0.006,0.996±0.001,0.984±0.003
6,Pareto,RankNet-Choice,0.612±0.007,0.624±0.026,0.772±0.029,0.060±0.010,0.877±0.011,0.672±0.014,0.971±0.006,0.891±0.019
7,Mode,FATE-Linear-Net,0.577±0.044,0.432±0.024,0.960±0.088,0.004±0.001,0.447±0.010,-0.000±0.001,0.500±0.002,0.563±0.005
8,Mode,GeneralizedLinearModel,0.597±0.000,0.442±0.000,0.999±0.001,0.003±0.000,0.443±0.000,0.000±0.000,0.497±0.004,0.561±0.002
9,Mode,RandomGuessing,0.597±0.000,0.442±0.000,1.000±0.000,0.003±0.000,0.442±0.000,0.000±0.000,0.500±0.000,0.442±0.000


In [34]:
import string
def get_val(val):
    vals =  [float(x) for x in re.findall(r"[-+]?\d*\.\d+|\d+", val)]
    return [vals[0], vals[0] - vals[1]*1e-3]
def mark_best(df):
    for col in list(df.columns)[1:]:
        values_str = df[['ChoiceModel',col]].as_matrix()
        values = np.array([get_val(val[1])for val in values_str])
        maxi = np.where(values[:,0] == values[:,0][np.argmax(values[:,0])])[0]
        for ind in maxi:
            values_str[ind] = [values_str[ind][0], "bfseries {}".format(values_str[ind][1])]
        df['ChoiceModel'] = values_str[:,0]
        df[col] = values_str[:,1]
    return df

In [35]:
import re
def get_name(name):
    if "Letor" in name:
        name = "Letor Year ${}$ Objects ${}$".format(name.split('_')[-3], name.split('_')[-1])
    if "Exp" in name:
        name = "Expedia Objects ${}$".format(name.split('_')[-1])
    return name

def create_latex(df):
    grouped = df.groupby(['Dataset'])
    for name, group in grouped:
        custom_dict = dict()
        for i, m in enumerate(models):
            custom_dict[m] = i
        group['rank'] = group['ChoiceModel'].map(custom_dict)
        group.sort_values(by='rank', inplace=True)
        del group["Dataset"]
        del group['rank']
        group = mark_best(group)
        group['ChoiceModel'].replace(to_replace=['GeneralizedLinearModel'], value='glm',inplace=True)
        group['ChoiceModel'].replace(to_replace=['FATE-Net'], value='fatenet',inplace=True)
        group['ChoiceModel'].replace(to_replace=['FETA-Net'], value='fetanet',inplace=True)
        group['ChoiceModel'].replace(to_replace=['RankNet-Choice'], value='ranknet',inplace=True)
        group['ChoiceModel'].replace(to_replace=['PairwiseSVM'], value='pairwisesvm',inplace=True)
        group['ChoiceModel'].replace(to_replace=['RandomGuessing'], value='random',inplace=True)
        group.rename(columns={'F1Score': '$F_1$-measure', 'Subset01Accuracy': 'Subset $0/1$ Accuracy', 'Aucscore':'Auc-Score'}, inplace=True)
        del group['Hammingaccuracy']
        del group['Precision']
        del group['Recall']
        #del group['Informedness']
        del group['Averageprecisionscore']
        print("############################################################################")
        print("Dataset {}\n".format(get_name(name)))
        latex_code = group.to_latex(index = False)
        latex_code = latex_code.replace(' ',"")
        latex_code = latex_code.replace('&'," & ")
        latex_code = str(latex_code)
        for learner in group['ChoiceModel']:
            latex_code = latex_code.replace(learner, "\\{}".format(learner))
        latex_code = latex_code.replace("bfseries", "\\{} ".format("bfseries"))
        latex_code = latex_code.replace("\\$", "$")
        latex_code = latex_code.replace("\\_", "_")
        print(latex_code)
for dataset in datasets:
    df = create_combined_dfs(dataset, latex_row=True)
    df.sort_values(by='Dataset')
    create_latex(df)

############################################################################
Dataset Pareto

\begin{tabular}{lllll}
\toprule
ChoiceModel & $F_1$-measure & Subset$0/1$Accuracy & Informedness & Auc-Score\\
\midrule
\fetanet & \bfseries 0.942(8) & \bfseries 0.680(28) & \bfseries 0.956(12) & \bfseries 0.999(0)\\
\fatenet & 0.913(9) & 0.506(37) & 0.911(6) & 0.996(1)\\
\ranknet & 0.612(7) & 0.060(10) & 0.672(14) & 0.971(6)\\
\pairwisesvm & 0.588(1) & 0.044(3) & 0.646(7) & 0.956(0)\\
\glm & 0.565(41) & 0.038(12) & 0.609(57) & 0.935(38)\\
\random & 0.232(0) & 0.000(0) & 0.000(0) & 0.500(0)\\
\bottomrule
\end{tabular}

############################################################################
Dataset Mode

\begin{tabular}{lllll}
\toprule
ChoiceModel & $F_1$-measure & Subset$0/1$Accuracy & Informedness & Auc-Score\\
\midrule
\fetanet & 0.809(5) & 0.311(32) & 0.695(9) & 0.981(6)\\
\fatenet & \bfseries 0.976(1) & \bfseries 0.883(10) & \bfseries 0.961(2) & \bfseries 0.992(1)\\
\ranknet & 0.597(0)

In [ ]:
grouped = df.groupby(['dataset'])
for name, group in grouped:
    df_path = os.path.join(DIR_PATH, 'results' , name.lower()+'.csv')
    group.to_csv(df_path)

In [ ]:
import numpy as np
np.arange(48,87)

X_train = np.arange(40).reshape(4,5,2)

learner_params = {}
learner_params['n_objects'], learner_params['n_object_features'] = X_train.shape[1:]

In [ ]:
"UNIQUE_MAX_OCCURRING".lower()